# Laboratorium 2
#### Bartosz Hanc

---

Zadanie polega na implementacji dwóch algorytmów kompresji:

- statycznego algorytmu Huffmana (2 p)
- dynamicznego algorytmu Huffmana (3 p)

Dla każdego z algorytmów należy wykonać następujące zadania:

1. Opracować format pliku przechowującego dane. Zwróć uwagę na dwie kwestie:
    - Liczba bitów wynikowego pliku nie musi być podzielna przez 8, ale z dysku zawsze odczytujemy
      pełne bajty, dlatego ważne jest, aby jakoś rozwiązać ten problem. W przeciwnym razie po
      dekompresji można uzyskać nadmiarowe dane. 
    - Plik wynikowy musi być binarny, tzn. rozwiązanie nie może zakładać, że w pliku tym zapisywane
      są 0 i 1 jako znaki ASCII.

2. Zaimplementować algorytm kompresji i dekompresji danych dla tego formatu pliku.

3. Zmierzyć współczynnik kompresji (wyrażone w procentach: 1 - plik_skompresowany /
   plik_nieskompresowany) dla plików o rozmiarach: 1kB, 10kB, 100kB, 1MB, o różnej zawartości:
    - wybrany przez Ciebie plik tekstowy z projektu Gutenberg,
    - wybrany przez Ciebie plik z kodem źródłowym jądra Linuksa,
    - plik ze znakami losowanymi z rozkładu jednostajnego - należy uwzględnić wszystkie 256
      wartości, a nie tylko znaki drukowalne.

W sumie w punkcie 3 należy przeprowadzić analizę dla łącznie 12 plików (4 rozmiary x 3 typy plików).

Zmierzyć czas kompresji i dekompresji dla plików z punktu 3.

In [1]:
from heapq import heapify, heappop, heappush
from bitarray import bitarray

In [43]:
class Node:
    def __init__(self, char, freq, left=None, right=None):
        self.char = char
        self.freq = freq
        self.left = left
        self.right = right

    def __gt__(self, other):
        return self.freq > other.freq


def build_tree(char_freq: dict) -> Node:
    nodes = [Node(char, char_freq[char]) for char in char_freq.keys()]
    heapify(nodes)

    while len(nodes) > 1:
        left = heappop(nodes)
        right = heappop(nodes)
        node = Node(None, left.freq + right.freq, left, right)
        heappush(nodes, node)

    if len(char_freq) == 1:
        char = list(char_freq.keys())[0]
        return Node(None, char_freq[char], Node(char, char_freq[char]), None)

    return nodes[0]


def generate_codes(tree_root: Node) -> dict:
    codes = {}

    def traverse(node, code=""):
        if node.char != None:
            codes[node.char] = code

        if node.left != None:
            traverse(node.left, code + "0")

        if node.right != None:
            traverse(node.right, code + "1")

    traverse(tree_root)

    return codes


def get_code(node: Node, char: str) -> str:
    find_code = ""

    def traverse(node, char, code=""):
        nonlocal find_code
        if node.char == char:
            find_code = code

        if node.left != None:
            traverse(node.left, char, code + "0")

        if node.right != None:
            traverse(node.right, char, code + "1")

    traverse(node, char)

    return find_code


def encode_static(text: str) -> tuple[bitarray, Node]:
    char_freq = {char: 0 for char in set(text)}
    for char in text:
        char_freq[char] += 1

    tree_root = build_tree(char_freq)
    codes = generate_codes(tree_root)
    encoded_text = bitarray()
    for char in text:
        encoded_text.extend(codes[char])

    return encoded_text, tree_root


def decode_static(input: bitarray, tree_root: Node) -> str:
    output = ""
    node = tree_root
    for bit in input:
        node = node.right if bit else node.left
        if node.char != None:
            output += node.char
            node = tree_root

    return output


def encode_dynamic(text: str, alphabet: set) -> bitarray:
    encoded_text = bitarray()
    char_freq = {char: 0 for char in alphabet}

    for char in text:
        node = build_tree(char_freq)
        encoded_text.extend(get_code(node, char))
        char_freq[char] += 1

    return encoded_text


def decode_dynamic(input: str, alphabet: set) -> str:
    output = ""
    char_freq = {char: 0 for char in alphabet}

    node = build_tree(char_freq)
    for bit in input:
        node = node.right if bit else node.left
        if node.char != None:
            output += node.char
            char_freq[node.char] += 1
            node = build_tree(char_freq)

    return output


#### Pomiary współczynnika kompresji i czasów działania

In [65]:
import os
import time


def runtests(infile: str, option="Static") -> None:
    outfile = infile + ".huff"
    with open(infile, "r", encoding="utf-8") as file:
        text = file.read()

    s = time.time()
    if option == "Dynamic":
        encoded_text = encode_dynamic(text, set(text))
    else:
        encoded_text, root = encode_static(text)
    e = time.time()
    encode_time = e - s

    s = time.time()
    if option == "Dynamic":
        decoded_text = decode_dynamic(encoded_text, set(text))
    else:
        decoded_text = decode_static(encoded_text, root)

    e = time.time()
    decode_time = e - s

    with open(outfile, "wb") as file:
        encoded_text.tofile(file)

    print(
        f"{text == decoded_text}\n",
        f"Compression ratio for file '{infile}':",
        "{:.3f} %\n".format(
            100 * (1 - os.path.getsize(outfile) / os.path.getsize(infile))
        ),
        "Compression time  : {:.3f} s\n".format(encode_time),
        "Decompression time: {:.3f} s\n".format(decode_time),
        "___________________\n",
    )


* Statyczny algorytm Huffmana

In [68]:
prefix = "test/"
for size in ("1kb", "10kb", "100kb", "1mb"):
    for sufix in ("-gutenberg", "-linux", "-uniform"):
        runtests(prefix + size + sufix + ".txt", "Static")


True
 Compression ratio for file 'test/1kb-gutenberg.txt': 46.514 %
 Compression time  : 0.000 s
 Decompression time: 0.001 s
 ___________________

True
 Compression ratio for file 'test/1kb-linux.txt': 36.626 %
 Compression time  : 0.001 s
 Decompression time: 0.003 s
 ___________________

True
 Compression ratio for file 'test/1kb-uniform.txt': 34.442 %
 Compression time  : 0.001 s
 Decompression time: 0.001 s
 ___________________

True
 Compression ratio for file 'test/10kb-gutenberg.txt': 46.643 %
 Compression time  : 0.002 s
 Decompression time: 0.007 s
 ___________________

True
 Compression ratio for file 'test/10kb-linux.txt': 39.012 %
 Compression time  : 0.003 s
 Decompression time: 0.007 s
 ___________________

True
 Compression ratio for file 'test/10kb-uniform.txt': 33.286 %
 Compression time  : 0.003 s
 Decompression time: 0.009 s
 ___________________

True
 Compression ratio for file 'test/100kb-gutenberg.txt': 46.756 %
 Compression time  : 0.023 s
 Decompression time: 0

* Dynamiczny algorytm Huffmana

In [69]:
prefix = "test/"
for size in ("1kb", "10kb", "100kb", "1mb"):
    for sufix in ("-gutenberg", "-linux", "-uniform"):
        runtests(prefix + size + sufix + ".txt", "Dynamic")

True
 Compression ratio for file 'test/1kb-gutenberg.txt': 43.625 %
 Compression time  : 0.095 s
 Decompression time: 0.070 s
 ___________________

True
 Compression ratio for file 'test/1kb-linux.txt': 33.113 %
 Compression time  : 0.762 s
 Decompression time: 0.679 s
 ___________________

True
 Compression ratio for file 'test/1kb-uniform.txt': -24.462 %
 Compression time  : 0.972 s
 Decompression time: 0.731 s
 ___________________

True
 Compression ratio for file 'test/10kb-gutenberg.txt': 46.156 %
 Compression time  : 1.191 s
 Decompression time: 0.911 s
 ___________________

True
 Compression ratio for file 'test/10kb-linux.txt': 37.874 %
 Compression time  : 2.549 s
 Decompression time: 1.911 s
 ___________________

True
 Compression ratio for file 'test/10kb-uniform.txt': 26.184 %
 Compression time  : 8.545 s
 Decompression time: 6.949 s
 ___________________

True
 Compression ratio for file 'test/100kb-gutenberg.txt': 46.707 %
 Compression time  : 11.792 s
 Decompression time: